# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [6]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [13]:
context = [ {'role':'system', 'content':"""

### Database schema (sqlite) + sample rows
CREATE TABLES employees (
    ID INTEGER PRIMARY KEY,
    name TEXT
    department_id INTEGER,
    salary REAL,
    hire_date DATE,
    FOREIGN KEY(department_id) REFERENCES departments(ID)
    );

-- SAMPLE ROWS (employees)
-- id |    name     | department_id | salary  | hire_date
-- 1001 | 'Alice'     |      1       | 70000.0 | 2015-3-1
-- 1002 | 'Bob'       |      2       | 80000.0 | 2016-7-15
-- 1003 | 'Charlie'   |      1       | 75000.0 | 2017-10-30
-- 1004 | 'Diana'     |      3       | 90000.0 | 2018-1-20
-- 1005 | 'Ethan'     |      2       | 85000.0 | 2019-5-25

CREATE TABLE salary (
    ID INTEGER PRIMARY KEY,
    employee_id INTEGER,
    year DATE,
    salary REAL,
    FOREIGN KEY(employee_id) REFERENCES employees(ID)
    );

-- SAMPLE ROWS (salary)
-- id | employee_id |   year    | salary
-- 1  |    1001     | 2019-12-31 | 70000.0
-- 2  |    1002     | 2019-12-31 | 80000.0
-- 3  |    1003     | 2019-12-31 | 75000.0
-- 4  |    1004     | 2019-12-31 | 90000.0
-- 5  |    1005     | 2019-12-31 | 85000.0

CREATE TABLE studies (
    ID INTEGER PRIMARY KEY,
    employee_id INTEGER,
    educational_level INTEGER,
    institution TEXT,
    years DATE,
    speciality TEXT,
    FOREIGN KEY(employee_id) REFERENCES employees(ID)
    );

-- SAMPLE ROWS (studies)
-- id | employee_id | educational_level | institution      |   years    |   speciality
-- 1  |    1001     |        3         | 'University A' | 2010-5-15 | 'Computer Science'
-- 2  |    1002     |        4         | 'University B' | 2011-6-20 | 'Business'
-- 3  |    1003     |        3         | 'University A' | 2012-7-25 | 'Mathematics'
-- 4  |    1004     |        5         | 'University C' | 2013-8-30 | 'Engineering'
-- 5  |    1005     |        4         | 'University B' | 2014-9-10 | 'Economics'
"""} ]

In [14]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
                 
-- Q: List all employee names and their departments.
-- SQL:
SELECT e.name, d.name AS department
FROM employees e
JOIN departments d ON d.id = e.department_id;

-- Q: Which employees were hired in 2023 or later?
-- SQL:
SELECT id, name, hiring_date
FROM employees
WHERE hiring_date >= '2023-01-01';

-- Q: Total salary per department (highest first).
-- SQL:
SELECT d.name AS department, SUM(e.salary) AS total_salary
FROM employees e
JOIN departments d ON d.id = e.department_id
GROUP BY d.name
ORDER BY total_salary DESC;

-- Q: Show employees and the projects they are assigned to (name only).
-- SQL:
SELECT e.name AS employee, p.name AS project
FROM employee_projects ep
JOIN employees e ON e.id = ep.employee_id
JOIN projects p ON p.id = ep.project_id;

-- Q: Count how many employees are in IT.
-- SQL:
SELECT COUNT(*) AS it_count
FROM employees e
JOIN departments d ON d.id = e.department_id
WHERE d.name = 'IT';
"""
})

In [15]:
#Function to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [19]:
#new
context_user = context.copy()
print(return_CCRMSQL("""List the names of all employees with a salary higher than 60000 that were hired after 2022 and studied Computer Science.""", context_user))

```sql
-- SQL:
SELECT e.name
FROM employees e
JOIN studies s ON e.id = s.employee_id
WHERE e.salary > 60000
AND e.hire_date > '2022-01-01'
AND s.speciality = 'Computer Science';
```


In [20]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("List the names of all employees with a salary higher than 60000 that were hired after 2022 and studied Computer Science.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE s.salary > 60000
AND s.year > '2022-01-01'
AND st.educational_level = 'Computer Science';
```

This SQL query retrieves the names of employees who have a salary higher than 60000, were hired after 2022, and studied Computer Science. It achieves this by joining the `employees`, `salary`, and `studies` tables on the corresponding ID columns and applying the specified conditions in the WHERE clause.


In [21]:
#new
print(return_CCRMSQL("List the names of all employees with a salary lower than 40000 that were hired before 1990 and did not study.", context_user))

```sql
-- SQL:
SELECT e.name
FROM employees e
LEFT JOIN studies s ON e.id = s.employee_id
WHERE e.salary < 40000 AND e.hire_date < '1990-01-01' AND s.id IS NULL;
```


In [22]:
#old
print(return_CCRMSQL("List the names of all employees with a salary lower than 40000 that were hired before 1990 and did not study.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
LEFT JOIN studies st ON e.ID_usr = st.ID_usr
WHERE s.salary < 40000 AND s.year < '1990-01-01' AND st.ID IS NULL;
```

This SQL query selects the names of employees from the "employees" table who have a salary lower than 40000, were hired before 1990, and did not have any records in the "studies" table. It uses JOIN and LEFT JOIN to combine the tables and filter the results accordingly.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?